# 七、认证加密
## 选择明文攻击安全的加密方案所面临的主动攻击
保密性保证了加密方案在面对选择明文攻击时具备语义安全，这种安全性只能防范监听者，无法应对主动攻击。

完整性保证了信息在面对选择信息攻击时，可以免于存在伪造。

而这一模块将更近一步，探讨如何防范主动的攻击者，也即具备篡改信息能力的攻击者，加密方案需要同时保证保密性和完整性，这时需要采用认证加密。

使用认证加密（保密性和完整性），或者MAC（完整性，不要求保密性），现实中只有这两种情况。

## 定义
认证加密的加密系统需要：1)保证CPA攻击下的语义安全。2)密文完整性，攻击者创造的新密文无法正确解密。这要求解密函数同时输出明文和是否完整的布尔逻辑服。

密文完整性的定义：攻击者发送新密文（不具备完整性），系统判定该密文完整的概率可忽略。

## 选择密文攻击
假定攻击者同时具备选择明文攻击（可以获得任意信息的密文）和选择密文攻击（可以获得任意密文对应的原文）的能力，攻击者希望破坏加密系统的语义安全性。

选择密文攻击的语义安全性：攻击者发送两条明文，系统返回一条密文（选择明文攻击）。攻击者无法直接判断密文对应哪一条明文（选择明文攻击的语义安全）。攻击>者继而在返回密文的基础上稍作修改，发送给系统，如果系统对新密文进行解密，攻击者可以从中获得相应信息，从而攻破密文对应的具体明文，则该系统不具备CCA攻击
的语义安全。

可以证明，认证加密确保了选择密文攻击的语义安全性。

认证加密的局限：1)无法阻止重复攻击（replay attack）。2)无法组织其它渠道攻击（时机）。

## 通过加密方案和讯息鉴别码来构建认证加密系统
在认证加密这个概念被正式引入之前，很多机构都各自组合加密方案与讯息鉴别码，不同的操作大致分为以下三类：

1)SSL，MAC-then-Encrypt：这种方案仅有在加密使用rand-CTR或者rand-CBC时，才能提供认证加密。CCM标准（CBC-MAC then CTR mode encryption）。

2)IPsec，Encrypt-then-MAC：这种方案被证明，无论使用何种加密和讯息鉴别码的组合，都始终能保证认证加密。GCM标准（CTR mode encryption then CW-MAC），EAX>标准（CTR encryption then CMAC）。

3)SSH，Encrypt-and-MAC：这种方案被证明无法提供认证加密，因为MAC只保证信息的完整性，对原文进行MAC之后的结果会泄露明文信息。

通行的三种标准CCM，GCM，EAX均支持AEAD（authenticated encryption with associated data，其中associated data只认证，不加密）。

## 案例分析：TLS
加密系统配备双向密钥，客户端向服务器、服务器向客户端，两种通信密钥不同。两端各自存有一个64位的计数器，会话开始时计数器开始计数，以此防范重复攻击。

加密过程：通信时，对应的单向密钥衍生为MAC密钥和加密密钥。首先计数器计数，对“计数器||头文件（类型、版本、长度）||数据”使用MAC密钥进行MAC得到标签，对头
文件||数据||标签进行补齐（补齐至AES模块长度），随机初始向量和加密密钥对其进行CBC加密。将头文件贴在密文之前。

解密过程：CBC解密，检查补齐的格式是否正确，如果不正确触发pad错误。检查对应的标签是否不正确，不正确触发MAC错误。

旧版本TLS的两个bug：1)CBC的初始向量IV可以预测。2)padding oracle：解密失败时，返回具体错误原因（pad错误或MAC错误）。

目前TLS依然面临头文件泄露信息长度的问题，这个问题没有很好的解决方案。

## CBC的补齐攻击
pad错误和mac错误有一个时间差，即使在新版的TLS中，系统不再返回具体的错误类型，攻击者依然可以利用这个漏洞。

Encrypt-then-MAC可以完全避免这个问题。

# 八、剩余的琐碎问题
## 密钥派生
如何由一个源密钥（source key，SK）派生出多个密钥（双向密钥，基于随机数的CBC所需要的多个密钥）？

利用密钥派生函数KDF，输入源密钥、环境变量CTX、所需密钥数量L，输出一串子密钥。分解来看，每个子密钥，从0到L，分别对其使用伪随机函数F。

伪随机函数需要保证输入的源密钥服从均匀分布，如果不符合这一情况，需要从源密钥中提取服从均匀分布的密钥，再将该密钥用KDF扩展成多个密钥。这个提取和扩展的
过程是确定的。

常见的方法所HKDF，用HMAC对源密钥进行提取，然后用HMAC作为KDF进行扩展。

基于密钥的KDF（PBKDF）：不要使用HMAC，因为密钥的熵通常不够，派生的密钥通常容易遭受字典攻击。有通用标准PKCS#5，对源密钥进行多次哈希。

## 确定性加密
假如加密数据存入数据库中，为便于查找，同一数据的多次加密结果需要是完全一致的，即需要确定性加密。

确定性加密的问题在于：1)如果攻击者看到两则相同的密文，他可以确定这两条信息的明文一致。当明文信息空间有限时，可能遭到攻击。2)确定性加密不可能具备选择>明文攻击的语义安全。

解决方案：加密系统从不对相同的信息进行二次加密，即（k，m）不重复。在现实情况中，可能明文信息空间无比巨大，或者信息架构上要求唯一性（比如唯一的用户id>）。

确定IV的CBC加密不具备确定性选择明文攻击的语义安全。

## 确定性加密系统的构建：SIV和wide PRP
1)合成初始向量（Synthetic IV）：两份密钥，一份用于通过明文和伪随机函数生成加密的随机部分（如IV），另一份用于加密。只要保证伪随机函数安全，则SIV不仅具
备选择明文攻击的安全性，也具备选择密文攻击的安全性，换言之，这是一个确定性认证加密。

2)仅使用PRP，再扩展：以AES为例，只能处理16字节的信息。可以用EME方法进行构造。具体见图。wide PRP同样提供了确定性认证加密。

## 可调加密
磁盘扇区是固定大小的，加密系统不能扩展明文，因此必须使用确定性加密，且不提供完整性。一个推论是，具备选择明文攻击安全的确定性加密系统，如果明文和密文>同长，那么加密系统必然是一个PRP。

设想用同样的密钥对每个扇区进行加密，则必然存在两个扇区明文相同，从而密文相同，造成信息泄露。

利用扇区编号生成每个扇区独立的PRP。XTS构造模式。

## 保留格式加密
信用卡卡号的加密结果应该和信用卡卡号同长。总计s种不同的卡号，需要构造一个{0, …, s-1}的PRP。2<sup>t-1</sup> < s <= 2<sup>t</sup>。构建Feistel网络。用>完成的PRP对所有输入进行操作，如果输出结果不在区间内，则再次调用PRP。
保留格式加密同样不提供完整性。